The task is based on the Fastai course, which uses the Kaggle competition [U.S. Patent Phrase to Phrase Matching](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/)



In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/input/us-patent-phrase-to-phrase-matching/test.csv


In [2]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
from pathlib import Path
path = Path('/kaggle/input/input/us-patent-phrase-to-phrase-matching')

Installing Huggingface datasets (if not already present)

In [4]:
!pip install -q datasets

In [5]:
#Python variables (here 'path') can be included using {} in bash
!ls {path}

sample_submission.csv  test.csv  train.csv


In [6]:
import pandas as pd
df = pd.read_csv(path/'train.csv')
#df.head()
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [7]:
#"Generates descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values"
print (df.describe())
print ()
print (df.describe(include='object'))

              score
count  36473.000000
mean       0.362062
std        0.258335
min        0.000000
25%        0.250000
50%        0.250000
75%        0.500000
max        1.000000

                      id                       anchor       target context
count              36473                        36473        36473   36473
unique             36473                          733        29340     106
top     8d135da0b55b8c88  component composite coating  composition     H01
freq                   1                          152           24    2186


In [8]:
#df.input = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor ###--> THis does not add input to df
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
df
#df.input.head()

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement
...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,TEXT1: B44; TEXT2: wooden article; ANC1: wood ...
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,TEXT1: B44; TEXT2: wooden box; ANC1: wood article
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,TEXT1: B44; TEXT2: wooden handle; ANC1: wood a...
36471,756ec035e694722b,wood article,wooden material,B44,0.75,TEXT1: B44; TEXT2: wooden material; ANC1: wood...


In [9]:
print(df.columns)
print(df.head())

Index(['id', 'anchor', 'target', 'context', 'score', 'input'], dtype='object')
                 id     anchor                  target context  score  \
0  37d61fd2272659b1  abatement  abatement of pollution     A47   0.50   
1  7b9652b17b68b7a4  abatement          act of abating     A47   0.75   
2  36d72442aefd8232  abatement         active catalyst     A47   0.25   
3  5296b0c19e1ce60e  abatement     eliminating process     A47   0.50   
4  54c1e3b9184cb5b6  abatement           forest region     A47   0.00   

                                               input  
0  TEXT1: A47; TEXT2: abatement of pollution; ANC...  
1  TEXT1: A47; TEXT2: act of abating; ANC1: abate...  
2  TEXT1: A47; TEXT2: active catalyst; ANC1: abat...  
3  TEXT1: A47; TEXT2: eliminating process; ANC1: ...  
4  TEXT1: A47; TEXT2: forest region; ANC1: abatement  


**Tokenization**

The Tokenization process converts strings to numbers to be used in NN. 

Huggingface's transformer object **Dataset** stores the input dataset.

"**Transformers** acts as the model-definition framework for state-of-the-art machine learning models in text, computer vision, audio, video, and multimodal models, for both inference and training." (https://huggingface.co/docs/transformers/en/index)

In [10]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

Two steps are involved

*Tokenization*: Split each text up into words (popularly, subwords are used, as here) as tokens.

*Numericalization*: Convert each word (or token) into a number.

Different Huggingface models(transformers) can be found here: https://huggingface.co/models

In [11]:
model_nm = 'microsoft/deberta-v3-small'

*AutoTokenizer* will create a tokenizer appropriate for a given pretrained model

In [12]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


An example of tokenizing a sentence. Further details on the process can be followed from the book "Deep Learning for Coders with Fastai and PyTorch" Chapter-10

In [13]:
tokz.tokenize("Checking if it is working properly.")

['▁Checking', '▁if', '▁it', '▁is', '▁working', '▁properly', '.']

In [14]:
#Tokenizing the inputs from ds; x is a document and x['input'] is the section we are willing to tokenize
#'ds.map' parallelize the process

def tok_func(x): return tokz(x["input"])

tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

*input_ids* is added to the dataset

In [15]:
print (tok_ds)
print ('First row')
tok_ds[0]

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})
First row


{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'score': 0.5,
 'input': 'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 'input_ids': [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [17]:
tokz.tokenize("TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement")


['▁TEXT',
 '1',
 ':',
 '▁A',
 '47',
 ';',
 '▁TEXT',
 '2',
 ':',
 '▁abatement',
 '▁of',
 '▁pollution',
 ';',
 '▁ANC',
 '1',
 ':',
 '▁abatement']

In [18]:
#To be noted, it's not a simple _
tokz.vocab['▁of'], tokz.vocab['▁TEXT']

(265, 54453)

Preparing labels: By default, *Transformers* uses the column name *labels* for targets, currently it is named as *score*. So need to rename it.

In [19]:
tok_ds = tok_ds.rename_columns({'score':'labels'})  #Can be run only once, can't be renamed twice
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

In [20]:
tok_ds[0]

{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'labels': 0.5,
 'input': 'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 'input_ids': [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

**Creating a validation set**

Higgingface creates training and validation sets using *train_test_split* and holds them in *DatasetDict* 

In [21]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

**Test Set**

In [22]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


In [23]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [24]:
#from sklearn.linear_model import LinearRegression
#from sklearn.preprocessing import PolynomialFeatures
#from sklearn.pipeline import make_pipeline

*np.corrcoef* presents all the (Pearson) correlation coefficients for every combination of columns in a dataset and generates a correlation coefficient matrix

In [25]:
def corr(x,y): return np.corrcoef(x,y)[0][1]

Huggingface expects a dict, where it (trainer) uses the keys of the dict to label the matric. Here labelled as 'pearson'

In [26]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

The validity of the model is measured with a metric (not same as loss function), here: 'Pearson coefficient' is used 

**Training our model**

In [27]:
from transformers import TrainingArguments,Trainer
bs = 128   #Batch size
epochs = 5
lr = 8e-5  #Learning rate

2025-08-07 12:42:31.119464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754570551.142687    1365 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754570551.150246    1365 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


*output_dir='outputs'* : Specifies the directory where the model checkpoints and training logs will be saved ('outputs')


*learning_rate=lr* : Sets the learning rate for the optimizer. 


*warmup_ratio=0.1* : Specifies the ratio of total training steps used for a linear warmup from 0 to the initial learning rate. Warmup helps in stabilizing the training process at the beginning.


*lr_scheduler_type='cosine'* : Defines the type of learning rate scheduler to use. A 'cosine' scheduler adjusts the learning rate following a cosine curve, which can help in fine-tuning the model by reducing the learning rate gradually.


*fp16=True* : Enables mixed-precision training using 16-bit floating-point numbers. This can speed up training and reduce memory usage on compatible hardware (like modern GPUs).


*eval_strategy="epoch"* : Determines when to evaluate the model during training. Setting it to "epoch" means the model will be evaluated at the end of each epoch.


*per_device_train_batch_size=bs* : Sets the batch size for training on each device (e.g., GPU). 


*per_device_eval_batch_size* : Sets the batch size for evaluation on each device. It is set to twice the training batch size to speed up the evaluation process


*num_train_epochs=epochs* : Specifies the total number of training epochs. 


*weight_decay=0.01* : Applies weight decay (L2 regularization) to the model weights. This helps in preventing overfitting by penalizing large weights.


*report_to='none'* : Disables logging to external tools like TensorBoard, Weights & Biases, etc. Setting it to 'none' means no training logs will be sent to these tools.

In [28]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    eval_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

'AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)': uses the pretrained model (model_nm) and retrieves the appropriate model architecture for sequence classification. *num_labels=1* defines a single output label, meaning the model is set up for a regression task or binary classification.

'Trainer()' handles the training and evaluation loop of the model. *tokenizer=tokz* defines the tokenizer to be used for preprocessing the text data. *compute_metrics=corr_d* defines the function (Pearson coefficient) that computes evaluation metrics during training. 

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1365/3597993663.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [30]:
trainer.train();

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.031425,0.770882
2,No log,0.031110,0.807556
3,No log,0.022745,0.822505
4,No log,0.023277,0.829488
5,0.030700,0.023717,0.831113


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [31]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.55440444],
       [ 0.67733246],
       [ 0.59256715],
       [ 0.390297  ],
       [-0.01822404],
       [ 0.61738127],
       [ 0.52325743],
       [ 0.0042911 ],
       [ 0.28748569],
       [ 1.0962193 ],
       [ 0.25615668],
       [ 0.2429643 ],
       [ 0.7715897 ],
       [ 0.99523842],
       [ 0.77812153],
       [ 0.45243955],
       [ 0.26934263],
       [-0.03151863],
       [ 0.63524711],
       [ 0.42111173],
       [ 0.43756419],
       [ 0.31096733],
       [ 0.1983379 ],
       [ 0.25154191],
       [ 0.60437584],
       [-0.03200336],
       [-0.01799715],
       [-0.03605735],
       [-0.0435862 ],
       [ 0.63208884],
       [ 0.30731422],
       [ 0.04474068],
       [ 0.68221527],
       [ 0.49376798],
       [ 0.43906361],
       [ 0.19034836]])

In [32]:
preds = np.clip(preds, 0, 1)
preds

array([[0.55440444],
       [0.67733246],
       [0.59256715],
       [0.390297  ],
       [0.        ],
       [0.61738127],
       [0.52325743],
       [0.0042911 ],
       [0.28748569],
       [1.        ],
       [0.25615668],
       [0.2429643 ],
       [0.7715897 ],
       [0.99523842],
       [0.77812153],
       [0.45243955],
       [0.26934263],
       [0.        ],
       [0.63524711],
       [0.42111173],
       [0.43756419],
       [0.31096733],
       [0.1983379 ],
       [0.25154191],
       [0.60437584],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.63208884],
       [0.30731422],
       [0.04474068],
       [0.68221527],
       [0.49376798],
       [0.43906361],
       [0.19034836]])